# IBM Applied Data Science Capstone Project - Week 5 - Final Notebook

## Opening a new Business in the city of Toronto

### -Build a dataframe of neighborhoods in Toronto, Canada by web scraping the data from Wikipedia page
### -Get the geographical coordinates of the Toronto neighborhoods
### -Obtain the venue data for the neighborhoods from Foursquare API
### -Create a table of different unique businesses/venues in Toronto with a numerical index corresponding to each business/venue
### -Prompt the entrepreneur to enter the numerical index of the business/venue matching with the new business/venue he is planning to set up
### -Explore and cluster the neighborhoods according to the business/venue prompted by user
### -Select/suggest the best cluster to open the new Business

## 1. Import the required Libraries

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
        #The display.max_columns option controls the number of columns to be printed. It receives an int or None("None" print all the columns)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

!conda install -c conda-forge lxml --yes

print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geopy-2.0.0                |     pyh9f0ad1d_0          63 KB  conda-forge
    certifi-2020.6.20          |   py36h9f0ad1d_0         151 KB  conda-forge
    openssl-1.1.1g             |       h516909a_0         2.1 MB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    ca-certificates-2020.6.20  |       hecda079_0         145 KB  conda-forge
    python_abi-3.6             |          1_cp36m           4 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.5 MB

The following NEW packages will be INSTALLED:

    geographiclib:   1.50-py_0          conda-forge
    geopy:           

## 2. Here I use pandas for scraping the page and read the table into a pandas dataframe.

### 2.1 Read the html page and store it in dataframe "df" and select the 1st table from df which is the required table.

In [2]:
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
df=pd.read_html(url,header=0)
df1=df[0]
df1

,Postal Code,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
7,M8A,Not assigned,Not assigned
8,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
9,M1B,Scarborough,"Malvern, Rouge"


###  

### 2.2 Drop all rows with a "Not assigned" Borough from the df1 data frame and save it as "df_tor" Dataframe

In [3]:
df2=df1.drop(df1[df1.Borough == 'Not assigned'].index)
df_tor=df2.reset_index(drop=True)
df_tor

,Postal Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
6,M1B,Scarborough,"Malvern, Rouge"
7,M3B,North York,Don Mills
8,M4B,East York,"Parkview Hill, Woodbine Gardens"
9,M5B,Downtown Toronto,"Garden District, Ryerson"


### 2.3 To check whether any row of df_tor Dataframe has a borough but a "Not assigned" neighbourhood.

In [4]:
df_check=df_tor[df_tor['Neighborhood']=='Not assigned']
df_check.shape[0]

0

### Thus NO row in df_tor Dataframe has a borough but a "Not assigned" neighbourhood

### 2.4 Displaying the cleaned and processed "df_tor" dataframe.

In [5]:
df_tor

,Postal Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
6,M1B,Scarborough,"Malvern, Rouge"
7,M3B,North York,Don Mills
8,M4B,East York,"Parkview Hill, Woodbine Gardens"
9,M5B,Downtown Toronto,"Garden District, Ryerson"


## 3. Load the Latitude and Longitudinal co-ordinates of Toronto neighbourhoods

In [6]:
filename = "http://cocl.us/Geospatial_data"
df3=pd.read_csv(filename)
df3

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
5,M1J,43.744734,-79.239476
6,M1K,43.727929,-79.262029
7,M1L,43.711112,-79.284577
8,M1M,43.716316,-79.239476
9,M1N,43.692657,-79.264848


## 4. Left merging df_tor and df3 dataframes to add Latitude and Longitude values to df_tor dataframe and name the merged dataframe as as "tor_cord".

In [7]:
tor_cord = pd.merge(df_tor, df3, on ='Postal Code', how ='left')

## 5. The final tor_cord dataframe with co-ordinates added

In [8]:
tor_cord

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village",43.667856,-79.532242
6,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
7,M3B,North York,Don Mills,43.745906,-79.352188
8,M4B,East York,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937


## 6. To Select only those boroughs that contain the word "Toronto"

In [9]:
tor_neigh=tor_cord.copy()
tor_neigh = tor_neigh[tor_neigh.Borough.str.contains("Toronto")]
tor_neigh = tor_neigh.reset_index(drop=True)
tor_neigh

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
4,M4E,East Toronto,The Beaches,43.676357,-79.293031
5,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306
6,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383
7,M6G,Downtown Toronto,Christie,43.669542,-79.422564
8,M5H,Downtown Toronto,"Richmond, Adelaide, King",43.650571,-79.384568
9,M6H,West Toronto,"Dufferin, Dovercourt Village",43.669005,-79.442259


In [10]:
print(tor_neigh.shape)
tor_neigh.head()

(39, 5)


,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
4,M4E,East Toronto,The Beaches,43.676357,-79.293031


### So 39 Boroughs have word 'Toronto' in their names

#  

## 7. Use geopy library to get the latitude and longitude values of Toronto

In [12]:
address = 'Toronto'

geolocator = Nominatim(user_agent="tor_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


## 8. Create a map of Toronto with neighborhoods superimposed on top

In [13]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(tor_neigh['Latitude'], tor_neigh['Longitude'], tor_neigh['Borough'], tor_neigh['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)

    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [14]:
# The code was removed by Watson Studio for sharing.

## 9. Let's explore the neighborhoods in our dataframe using Foursquare API

### 9.1 Define a function get_category_type that extracts the category of the venue

In [15]:
def get_category_type(row):
    try:
        categories_list = row['categories'] #select 'categories' column value of the row. Each value is a list
    except:
        categories_list = row['venue.categories'] #select 'venue.categories' column value of the row. Each value is a list
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name'] #returns value of "name" keyword from first element(which is a dictionary) of "categories_list" 
                                          #list

### 9.2 Define a function getNearbyVenues to find the venues in all the neighborhoods in Toronto

In [16]:
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 1000 # define radius

def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])
        
    #Create a new Dataframe nearby_venues from venues_list
    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
            #first it will select each tuple venue_list from "venues_list" list and then select each element "item" of "venue_list" tuple
               #and add them as row elements(same row but successive columns) in the "nearby_venues" dataframe.
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

## 10. Code to run the function getNearbyVenues on each neighborhood and create a new dataframe called toronto_venues.

In [17]:
toronto_venues = getNearbyVenues(names=tor_neigh['Neighborhood'],
                                   latitudes=tor_neigh['Latitude'],
                                   longitudes=tor_neigh['Longitude']
                                  )

Regent Park, Harbourfront
Queen's Park, Ontario Provincial Government
Garden District, Ryerson
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Richmond, Adelaide, King
Dufferin, Dovercourt Village
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
The Danforth West, Riverdale
Toronto Dominion Centre, Design Exchange
Brockton, Parkdale Village, Exhibition Place
India Bazaar, The Beaches West
Commerce Court, Victoria Hotel
Studio District
Lawrence Park
Roselawn
Davisville North
Forest Hill North & West, Forest Hill Road Park
High Park, The Junction South
North Toronto West, Lawrence Park
The Annex, North Midtown, Yorkville
Parkdale, Roncesvalles
Davisville
University of Toronto, Harbord
Runnymede, Swansea
Moore Park, Summerhill East
Kensington Market, Chinatown, Grange Park
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
R

### 10.1 Let's check the size of the resulting dataframe

In [18]:
print(toronto_venues.shape)
toronto_venues.head()

(1617, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Regent Park, Harbourfront",43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
1,"Regent Park, Harbourfront",43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,"Regent Park, Harbourfront",43.65426,-79.360636,Cooper Koo Family YMCA,43.653249,-79.358008,Distribution Center
3,"Regent Park, Harbourfront",43.65426,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa
4,"Regent Park, Harbourfront",43.65426,-79.360636,Dominion Pub and Kitchen,43.656919,-79.358967,Pub


## 11. Display how many venues were returned for each neighborhood

In [19]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Berczy Park,58,58,58,58,58,58
"Brockton, Parkdale Village, Exhibition Place",22,22,22,22,22,22
"Business reply mail Processing Centre, South Central Letter Processing Plant Toronto",15,15,15,15,15,15
"CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport",15,15,15,15,15,15
Central Bay Street,66,66,66,66,66,66
Christie,18,18,18,18,18,18
Church and Wellesley,74,74,74,74,74,74
"Commerce Court, Victoria Hotel",100,100,100,100,100,100
Davisville,33,33,33,33,33,33


#  

## 12. To find out how many unique categories can be curated from all the returned venues.

In [20]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

toronto_venues['Venue Category'].unique()

There are 234 uniques categories.


array(['Bakery', 'Coffee Shop', 'Distribution Center', 'Spa', 'Pub',
       'Park', 'Restaurant', 'Breakfast Spot', 'Gym / Fitness Center',
       'Historic Site', 'Farmers Market', 'Performing Arts Venue',
       'Chocolate Shop', 'Dessert Shop', 'French Restaurant', 'Café',
       'Yoga Studio', 'Theater', 'Event Space', 'Shoe Store',
       'Ice Cream Shop', 'Art Gallery', 'Cosmetics Shop',
       'Asian Restaurant', 'Electronics Store', 'Bank', 'Beer Store',
       'Health Food Store', 'Wine Shop', 'Antique Shop',
       'Italian Restaurant', 'Sushi Restaurant', 'Creperie', 'Beer Bar',
       'Arts & Crafts Store', 'Burrito Place', 'Mexican Restaurant',
       'Hobby Shop', 'Diner', 'Fried Chicken Joint', 'Discount Store',
       'Smoothie Shop', 'Sandwich Place', 'Gym', 'Bar',
       'College Auditorium', 'Comic Shop', 'Clothing Store', 'Tea Room',
       'Plaza', 'Music Venue', 'Pizza Place', 'Thai Restaurant',
       'Ramen Restaurant', 'College Rec Center', 'Sporting Goods Shop

## 13. Analyze Each Neighborhood

In [21]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

            
# add neighborhood column back to dataframe
toronto_onehot['Neighborhoods'] = toronto_venues['Neighborhood']

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Neighborhoods,Afghan Restaurant,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Board Shop,Boat or Ferry,Bookstore,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Convention Center,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Home Service,Hookah Bar,Hospital,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts Dojo,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Outdoor Sculpture,Park,Performing Arts Venue,Pet Store,Pharmacy,Photography Studio,Pizza Place,Plane,Playground,Plaza,Poke Place,Post Office,Poutine Place,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,School,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soup Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Women's Store,Yoga Studio
0,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,

In [22]:
toronto_onehot.shape

(1617, 235)

## 14. Group rows by neighborhood and by taking the mean of the frequency of occurrence of each category.

In [23]:
toronto_grouped = toronto_onehot.groupby('Neighborhoods').mean().reset_index()
print(toronto_grouped.shape)
toronto_grouped

(39, 235)


,Neighborhoods,Afghan Restaurant,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Board Shop,Boat or Ferry,Bookstore,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Convention Center,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Home Service,Hookah Bar,Hospital,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts Dojo,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Outdoor Sculpture,Park,Performing Arts Venue,Pet Store,Pharmacy,Photography Studio,Pizza Place,Plane,Playground,Plaza,Poke Place,Post Office,Poutine Place,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,School,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soup Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Women's Store,Yoga Studio
0,Berczy Park,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.017241,0.000000,0.000000,0.000000,0.000000,0.000000,0.017241,0.000000,0.017241,0.034483,0.000000,0.000000,0.00,0.017241,0.017241,0.000000,0.034483,0.000000,0.000000,0.017241,0.000000,0.000000,0.000000,0.000000,0.017241,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.034483,0.00,0.000000,0.000000,0.000000,0.034483,0.000000,0.000000,0.000000,0.000000,0.017241,0.051724,0.086207,0.000000,0.000000,0.000000,0.00,0.000000,0.017241,0.000000,0.017241,0.000000,0.00,0.000000,0.000,0.017241,0.000000,0.000000,0.000000,0.000000,0.017241,0.000000,0.

## 15. Print each neighborhood along with the top 5 most common venues.

In [24]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhoods']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhoods'] == hood].T.reset_index() #select each row of toronto_grouped and transpose it
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:] #select rows starting from Row 1, ie, drop Row 0(which is the Neighborhoods row)
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2}) # round 'freq' column values of temp dataframe to 2 digits.
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Berczy Park----
          venue  freq
0   Coffee Shop  0.09
1  Cocktail Bar  0.05
2      Beer Bar  0.03
3   Cheese Shop  0.03
4          Café  0.03


----Brockton, Parkdale Village, Exhibition Place----
                venue  freq
0                Café  0.14
1      Breakfast Spot  0.09
2         Coffee Shop  0.09
3  Italian Restaurant  0.05
4             Stadium  0.05


----Business reply mail Processing Centre, South Central Letter Processing Plant Toronto----
                  venue  freq
0           Yoga Studio  0.07
1         Auto Workshop  0.07
2  Gym / Fitness Center  0.07
3    Light Rail Station  0.07
4         Garden Center  0.07


----CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport----
              venue  freq
0    Airport Lounge  0.13
1   Airport Service  0.13
2  Airport Terminal  0.13
3       Coffee Shop  0.07
4   Harbor / Marina  0.07


----Central Bay Street----
                 venue  freq
0          Coffee S

In [25]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 234 uniques categories.


## 16. Create a Table of all the Unique Venues/Businesses Categories in Toronto with an index number associated to each venue/business category.

In [26]:
data = toronto_venues['Venue Category'].unique()
list_of_business = pd.DataFrame(data,columns=['Business/Venue Category'])
list_of_business.sort_values(["Business/Venue Category"], inplace=True)
list_of_business = list_of_business.reset_index(drop=True)
list_of_business.index.name = 'Business Index No:'
list_of_business

,Business/Venue Category
Business Index No:,
0,Afghan Restaurant
1,Airport
2,Airport Food Court
3,Airport Lounge
4,Airport Service
5,Airport Terminal
6,American Restaurant
7,Antique Shop
8,Aquarium


## 17. Define a function to list the selected business and return a Dataframe containing the Neighbourhoods and the selected business frequency in each neighbourhood, based on the Business Index No selected by the user.

In [27]:
def business_name(category):
    busi_name = list_of_business.iloc[category][0]
    print("The selected Business is:",busi_name)
    tor_new_busi = toronto_grouped[["Neighborhoods",busi_name]]
    return tor_new_busi

## 18. Prompt the user to enter the Venue/Business category Index No: from the above table that matches to the Business he wishes to open.

### 18.1 Suppose an entrepreneur wishes to open an Italian Restaurant. From the Business Index Table, the index no of Italian Restaurant is 132 and so all the user needs to do is to input 132 when prompted.

In [45]:
business_cat = int(input("Enter the Business Index No: corresponding to the Business you wish to open by referring the list_of_business table:"))
tor_new_business = business_name(business_cat)
tor_new_business.head()

Enter the Business Index No: corresponding to the Business you wish to open by referring the list_of_business table:132
The selected Business is: Italian Restaurant


,Neighborhoods,Italian Restaurant
0,Berczy Park,0.000000
1,"Brockton, Parkdale Village, Exhibition Place",0.045455
2,"Business reply mail Processing Centre, South C...",0.000000
3,"CN Tower, King and Spadina, Railway Lands, Har...",0.000000
4,Central Bay Street,0.060606


### Thus tor_new_business dataframe is created which contains the details of the Business entered by user, here for eg: Italian Restaurant.

#  

## 19. Cluster Neighborhoods: Use K-Means to cluster the neighborhood into 3 clusters based on the Business selected by user.

In [46]:
# set number of clusters
kclusters = 3

tor_new_business_clustering = tor_new_business.drop('Neighborhoods', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(tor_new_business_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_

array([0, 1, 0, 0, 1, 1, 0, 2, 1, 0, 0, 0, 0, 2, 2, 2, 1, 0, 0, 2, 0, 0,
       1, 2, 0, 0, 0, 0, 1, 2, 1, 2, 2, 0, 0, 0, 1, 2, 1], dtype=int32)

## 20. Create a new dataframe 'tor_new_business_merged' that includes the Cluster Labels for each neighborhood.

In [47]:
tor_new_business_merged = tor_new_business.copy()
tor_new_business_merged["Cluster Labels"] = kmeans.labels_
tor_new_business_merged.rename(columns={"Neighborhoods": "Neighborhood"}, inplace=True)
tor_new_business_merged

,Neighborhood,Italian Restaurant,Cluster Labels
0,Berczy Park,0.000000,0
1,"Brockton, Parkdale Village, Exhibition Place",0.045455,1
2,"Business reply mail Processing Centre, South C...",0.000000,0
3,"CN Tower, King and Spadina, Railway Lands, Har...",0.000000,0
4,Central Bay Street,0.060606,1
5,Christie,0.055556,1
6,Church and Wellesley,0.000000,0
7,"Commerce Court, Victoria Hotel",0.030000,2
8,Davisville,0.060606,1
9,Davisville North,0.000000,0


In [48]:
tor_new_business_merged.shape

(39, 3)

## 21. Create a new dataframe 'tor_neigh_filt' that contains the Latitude and Longitudinal values for each neighborhood.

In [49]:
tor_neigh_filt = tor_neigh[["Neighborhood","Latitude","Longitude"]]
tor_neigh_filt.head()

,Neighborhood,Latitude,Longitude
0,"Regent Park, Harbourfront",43.654260,-79.360636
1,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
2,"Garden District, Ryerson",43.657162,-79.378937
3,St. James Town,43.651494,-79.375418
4,The Beaches,43.676357,-79.293031


## 22. Merge 'tor_new_business_merged' Dataframe with 'tor_neigh_filt' Dataframe to add Latitude and Longitudinal values for each neighborhood in the original 'tor_new_business_merged' Dataframe.

In [50]:
tor_new_business_merged = tor_new_business_merged.join(tor_neigh_filt.set_index("Neighborhood"), on="Neighborhood")
print(tor_new_business_merged.shape)
tor_new_business_merged 

(39, 5)


,Neighborhood,Italian Restaurant,Cluster Labels,Latitude,Longitude
0,Berczy Park,0.000000,0,43.644771,-79.373306
1,"Brockton, Parkdale Village, Exhibition Place",0.045455,1,43.636847,-79.428191
2,"Business reply mail Processing Centre, South C...",0.000000,0,43.662744,-79.321558
3,"CN Tower, King and Spadina, Railway Lands, Har...",0.000000,0,43.628947,-79.394420
4,Central Bay Street,0.060606,1,43.657952,-79.387383
5,Christie,0.055556,1,43.669542,-79.422564
6,Church and Wellesley,0.000000,0,43.665860,-79.383160
7,"Commerce Court, Victoria Hotel",0.030000,2,43.648198,-79.379817
8,Davisville,0.060606,1,43.704324,-79.388790
9,Davisville North,0.000000,0,43.712751,-79.390197


## 23. Clean the 'tor_new_business_merged' dataframe to sort the neighbourhoods based on Cluster Labels.

In [51]:
tor_new_business_merged.sort_values(["Cluster Labels"], inplace=True)
tor_new_business_merged.reset_index(drop=True)

,Neighborhood,Italian Restaurant,Cluster Labels,Latitude,Longitude
0,Berczy Park,0.000000,0,43.644771,-79.373306
1,"Regent Park, Harbourfront",0.000000,0,43.654260,-79.360636
2,Lawrence Park,0.000000,0,43.728020,-79.388790
3,"Kensington Market, Chinatown, Grange Park",0.000000,0,43.653206,-79.400049
4,"Richmond, Adelaide, King",0.000000,0,43.650571,-79.384568
5,Rosedale,0.000000,0,43.679563,-79.377529
6,Roselawn,0.000000,0,43.711695,-79.416936
7,"Forest Hill North & West, Forest Hill Road Park",0.000000,0,43.696948,-79.411307
8,"First Canadian Place, Underground city",0.010000,0,43.648429,-79.382280
9,"Moore Park, Summerhill East",0.000000,0,43.689574,-79.383160


## 24. Visualize the resulting clusters using Folium map.

In [52]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(tor_new_business_merged['Latitude'], tor_new_business_merged['Longitude'], tor_new_business_merged['Neighborhood'], tor_new_business_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' - Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## 25. Examine Clusters.

### 25.1 Cluster 0

In [53]:
clus_0 = tor_new_business_merged.loc[tor_new_business_merged['Cluster Labels'] == 0]
clus_0

,Neighborhood,Italian Restaurant,Cluster Labels,Latitude,Longitude
0,Berczy Park,0.00,0,43.644771,-79.373306
24,"Regent Park, Harbourfront",0.00,0,43.654260,-79.360636
18,Lawrence Park,0.00,0,43.728020,-79.388790
17,"Kensington Market, Chinatown, Grange Park",0.00,0,43.653206,-79.400049
25,"Richmond, Adelaide, King",0.00,0,43.650571,-79.384568
26,Rosedale,0.00,0,43.679563,-79.377529
27,Roselawn,0.00,0,43.711695,-79.416936
12,"Forest Hill North & West, Forest Hill Road Park",0.00,0,43.696948,-79.411307
11,"First Canadian Place, Underground city",0.01,0,43.648429,-79.382280
20,"Moore Park, Summerhill East",0.00,0,43.689574,-79.383160


### 25.2 Cluster 1

In [54]:
clus_1 = tor_new_business_merged.loc[tor_new_business_merged['Cluster Labels'] == 1]
clus_1

,Neighborhood,Italian Restaurant,Cluster Labels,Latitude,Longitude
30,"St. James Town, Cabbagetown",0.044444,1,43.667967,-79.367675
28,"Runnymede, Swansea",0.050000,1,43.651571,-79.484450
36,"The Danforth West, Riverdale",0.069767,1,43.679557,-79.352188
38,"University of Toronto, Harbord",0.055556,1,43.662696,-79.400049
16,"India Bazaar, The Beaches West",0.047619,1,43.668999,-79.315572
8,Davisville,0.060606,1,43.704324,-79.388790
5,Christie,0.055556,1,43.669542,-79.422564
4,Central Bay Street,0.060606,1,43.657952,-79.387383
1,"Brockton, Parkdale Village, Exhibition Place",0.045455,1,43.636847,-79.428191
22,"Parkdale, Roncesvalles",0.066667,1,43.648960,-79.456325


### 25.3 Cluster 2

In [55]:
clus_2 = tor_new_business_merged.loc[tor_new_business_merged['Cluster Labels'] == 2]
clus_2

,Neighborhood,Italian Restaurant,Cluster Labels,Latitude,Longitude
23,"Queen's Park, Ontario Provincial Government",0.030303,2,43.662301,-79.389494
37,"Toronto Dominion Centre, Design Exchange",0.030000,2,43.647177,-79.381576
15,"High Park, The Junction South",0.040000,2,43.661608,-79.464763
14,"Harbourfront East, Union Station, Toronto Islands",0.030000,2,43.640816,-79.381752
13,"Garden District, Ryerson",0.020000,2,43.657162,-79.378937
29,St. James Town,0.025641,2,43.651494,-79.375418
31,Stn A PO Boxes,0.030928,2,43.646435,-79.374846
32,Studio District,0.025000,2,43.659526,-79.340923
7,"Commerce Court, Victoria Hotel",0.030000,2,43.648198,-79.379817
19,"Little Portugal, Trinity",0.022727,2,43.647927,-79.419750


## 26. To Automatically suggest the Best Cluster using the Mean value of the frequency of occurrence of the selected Business in each Cluster.

### 26.1 To find the mean value of frequency of occurence of the selected business in each cluster and assign them to k0,k1 and k2 for Clusters 0,1 and 2 respectively.

In [56]:
k0 = clus_0.iloc[:,1].mean()
k1 = clus_1.iloc[:,1].mean()
k2 = clus_2.iloc[:,1].mean()
print("Mean value of frequency of occurence of"+" "+tor_new_business_merged.columns[1]+" "+"in Cluster 0 is k0 =",k0)
print("Mean value of frequency of occurence of"+" "+tor_new_business_merged.columns[1]+" "+"in Cluster 1 is k1 =",k1)
print("Mean value of frequency of occurence of"+" "+tor_new_business_merged.columns[1]+" "+"in Cluster 2 is k2 =",k2)

Mean value of frequency of occurence of Italian Restaurant in Cluster 0 is k0 = 0.0005263157894736842
Mean value of frequency of occurence of Italian Restaurant in Cluster 1 is k1 = 0.05562753783684016
Mean value of frequency of occurence of Italian Restaurant in Cluster 2 is k2 = 0.0284599163722875


### 26.2 Save the mean values in a list called clus_list.

In [57]:
mean_list=[k0,k1,k2]
mean_list

[0.0005263157894736842, 0.05562753783684016, 0.0284599163722875]

### 26.3 Define a function 'clus_choice' to recommend the Best cluster and the second best cluster to start the Business

In [58]:
def clus_choice(list):
    choice_df = pd.DataFrame(list,columns=['Cluster Mean'])
    choice_df.sort_values(["Cluster Mean"], inplace=True)
    a = choice_df.index.values
    clus_list = a.tolist()
    print("The Best Cluster to start"+" "+tor_new_business_merged.columns[1]+" "+"in Toronto is Cluster:",clus_list[0])
    print("The Second best cluster to start"+" "+tor_new_business_merged.columns[1]+" "+"in Toronto is Cluster:",clus_list[1])

### 26.4 Call the function 'clus_choice' to display the Best and second best clusters to start the business, here for eg. Italian Restaurant.

In [59]:
clus_choice(mean_list)

The Best Cluster to start Italian Restaurant in Toronto is Cluster: 0
The Second best cluster to start Italian Restaurant in Toronto is Cluster: 2


## Observations:

#### 1. As we can see from the Clusters, for the case of Italian Restaurant selected by the user/entrepreneur, most of them are concentrated in neighborhoods in Cluster 1 and the least concentration is observed in neighborhoods in Cluster 0.  Neighborhoods in Cluster 2 seems to have a medium concentration of Italian restaurants. So looking at nearby venues, it seems any neighborhood in Cluster 0 might be a good location as there are very few Italian restaurants in these areas. 
#### 2. Thus we also verify that this analysis exactly matches with the predictions of the "clus_choice" function which automatically suggests the Best and second best Clusters to start the business.
#### 3. Therefore, this project recommends the entrepreneurs to open a new Italian restaurant in neighborhoods in cluster 0 with little to no competition as they clearly get an early mover advantage. Entrepreneurs with unique selling propositions who stand out from the competition can also think about opening new Italian restaurant in neighborhoods in cluster 2 with moderate competition. Lastly, entrepreneurs are advised to avoid neighborhoods in cluster 1 which already have high concentration of Italian restaurants and suffering from intense competition.